In [2]:
import requests, json, itertools
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta

pd.options.display.max_rows =  500

In [3]:
def get_all_stations():
    r = requests.get(
        "https://www.maxjeune-tgvinoui.sncf/api/public/refdata/freeplaces-stations",
        timeout=30,
    )
    stations = {}
    for station in r.json()["stations"]:
        stations[station["station"]] = station["codeStation"]
    return stations

In [4]:
def lookup_one_day(orig: list, dest: list, day: str) -> pd.DataFrame:
    results = []
    for station_tuple in itertools.product(*[orig, dest]):
        r = requests.post(
            "https://www.maxjeune-tgvinoui.sncf/api/public/refdata/search-freeplaces-proposals",
            json={
                "departureDateTime": day,
                "destination": station_tuple[1],
                "origin": station_tuple[0],
            },
            timeout=30,
        )
        try:
            print(r)
            for proposal in r.json()["proposals"]:
                result = {}
                result["start_time"] = datetime.strptime(
                    proposal["departureDate"], "%Y-%m-%dT%H:%M"
                )
                result["date"] = result["start_time"].date()
                result["day"] = result["start_time"].strftime("%A")
                result["hour"] = result["start_time"].hour
                result["number"] = proposal["trainNumber"]
                result["orig"] = proposal["origin"]["label"]
                result["dest"] = proposal["destination"]["label"]
                result["end_time"] = datetime.strptime(
                    proposal["arrivalDate"], "%Y-%m-%dT%H:%M"
                )
                result["seats"] = proposal["freePlaces"]
                result["type"] = proposal["trainEquipment"]
                result["duration"] = round((result["end_time"].timestamp() - result["start_time"].timestamp())/3600, 2)
                result["ts_start"] = result["start_time"].timestamp()*1000
                result["ts_end"] = result["end_time"].timestamp()*1000
                results.append(result)
        except KeyError:
            pass
    return pd.DataFrame.from_dict(results)

In [30]:
r = requests.post(
    "https://www.maxjeune-tgvinoui.sncf/api/public/refdata/search-freeplaces-proposals",
    json={
        "departureDateTime": "2024-03-06",
        "destination": "FRNTE",
        "origin": "FRDJU",
    },
    timeout=30,
    headers={
        'authority': 'www.maxjeune-tgvinoui.sncf',
        'accept': 'application/json',
        'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'content-type': 'application/json',
        'Cookie': 'datadome=Si0MAjsVe3L4PbQIJHtGyYRhHX98l~oVX6ya4C_PZPcbHVgWF8yApj8Ugyg3h8p4vqQTMyLUavXjYjKt3B0remJLKuDLp_1KqN8xmF58r8Ruexwg69ie_4LxZ0ToH7HV; didomi_token=euJ1c2VyX2lkIjoiMThlMGU2MmEtY2YwYS02Yjg5LWEzNjUtMmIwZjQ2YmE4NmE3IiwiY3JlYXRlZCI6IjIwMjQtMDMtMDVUMTE6MzI6MTEuNjQ2WiIsInVwZGF0ZWQiOiIyMDI0LTAzLTA1VDExOjMyOjEyLjUzOFoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiYzptaWQiLCJjOmRhdGFkb2ciXX0sInB1cnBvc2VzIjp7ImVuYWJsZWQiOlsibW9uaXRvcmluZyIsImFuYWx5dGljcyJdfSwidmVyc2lvbiI6Mn0=;',
        'origin': 'https://www.maxjeune-tgvinoui.sncf',
        'referer': 'https://www.maxjeune-tgvinoui.sncf/sncf-connect/max-planner',
        'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'x-client-app': 'MAX_JEUNE',
        'x-client-app-version': '1.46.1',
    }
)

In [31]:
r.content

b'{"updatedAt":1709639328278,"expiresAt":1709639628278,"freePlacesRatio":0.44,"proposals":[{"freePlaces":77,"origin":{"rrCode":"FRDJU","label":"MASSY TGV"},"destination":{"rrCode":"FRNTE","label":"NANTES"},"departureDate":"2024-03-06T06:34","trainNumber":"8863","trainEquipment":"INOUI","arrivalDate":"2024-03-06T08:54"},{"freePlaces":70,"origin":{"rrCode":"FRDJU","label":"MASSY TGV"},"destination":{"rrCode":"FRNTE","label":"NANTES"},"departureDate":"2024-03-06T08:37","trainNumber":"5351","trainEquipment":"INOUI","arrivalDate":"2024-03-06T11:11"},{"freePlaces":19,"origin":{"rrCode":"FRDJU","label":"MASSY TGV"},"destination":{"rrCode":"FRNTE","label":"NANTES"},"departureDate":"2024-03-06T12:35","trainNumber":"5354","trainEquipment":"INOUI","arrivalDate":"2024-03-06T14:50"},{"freePlaces":47,"origin":{"rrCode":"FRDJU","label":"MASSY TGV"},"destination":{"rrCode":"FRNTE","label":"NANTES"},"departureDate":"2024-03-06T20:35","trainNumber":"5358","trainEquipment":"INOUI","arrivalDate":"2024-03-

In [7]:
def lookup_date_range_one_way(origs, dests, start, end):
    start = datetime.strptime(start, "%Y-%m-%d").date()
    end = datetime.strptime(end, "%Y-%m-%d").date()

    results = []
    current_date = start
    while current_date <= end:
        results.append(lookup_one_day(origs, dests, current_date.strftime('%Y-%m-%d')))
        current_date += timedelta(days=1)
    
    return pd.concat(results).sort_values(by='start_time')

In [8]:
def lookup_date_range_both_ways(stationsA, stationsB, start, end):
    a_to_b = lookup_date_range_one_way(stationsA, stationsB, start, end)
    a_to_b.insert(0, 'direction', 'outbound')
    b_to_a = lookup_date_range_one_way(stationsB, stationsA, start, end)
    b_to_a.insert(0, 'direction', 'inbound')
    return pd.concat([a_to_b, b_to_a]).sort_values(by='start_time')

In [11]:
result = lookup_date_range_both_ways(["FRDJU"], ["FRNTE"], '2024-03-06', '2024-03-10').reset_index().drop('index', axis=1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
result.head(20)

,direction,start_time,date,day,hour,number,orig,dest,end_time,seats,type,duration,ts_start,ts_end
0,inbound,2023-09-16 04:53:00,2023-09-16,Saturday,4,5300,NANTES,MASSY TGV,2023-09-16 07:18:00,20,INOUI,2.42,1.694833e+12,1.694841e+12
1,inbound,2023-09-16 06:01:00,2023-09-16,Saturday,6,9886,NANTES,MASSY TGV,2023-09-16 08:18:00,8,TGV,2.28,1.694837e+12,1.694845e+12
2,outbound,2023-09-16 08:46:00,2023-09-16,Saturday,8,5351,MASSY TGV,NANTES,2023-09-16 11:19:00,7,INOUI,2.55,1.694847e+12,1.694856e+12
3,outbound,2023-09-16 10:42:00,2023-09-16,Saturday,10,5486,MASSY TGV,NANTES,2023-09-16 13:19:00,7,INOUI,2.62,1.694854e+12,1.694863e+12
4,outbound,2023-09-16 12:12:00,2023-09-16,Saturday,12,5224,MASSY TGV,NANTES,2023-09-16 14:44:00,5,INOUI,2.53,1.694859e+12,1.694868e+12
5,outbound,2023-09-16 12:41:00,2023-09-16,Saturday,12,5354,MASSY TGV,NANTES,2023-09-16 14:50:00,13,INOUI,2.15,1.694861e+12,1.694869e+12
6,inbound,2023-09-16 12:44:00,2023-09-16,Saturday,12,5306,NANTES,MASSY TGV,2023-09-16 15:18:00,35,INOUI,2.57,1.694861e+12,1.694870e+12
7,inbound,2023-09-16 14:05:00,2023-09-16,Saturday,14,5480,NANTES,MASSY TGV,2023-09-16 16:19:00,4,INOUI,2.23,1.694866e+12,1.694874e+12
8,inbound,2023-09-16 15:18:00,2023-09-16,Saturday,15,5288,NANTES,MASSY TGV,2023-09-16 17:47:00,32,INOUI,2.48,1.694870e+12,1.694879e+12
9,inbound,2023-09-16 17:04:00,2023-09-16,Saturday,17,5320,NANTES,MASSY TGV,2023-09-16 19:21:00,52,INOUI,2.28,1.694877e+12,1.694885e+12


In [11]:
rg = result.groupby(['date', 'day', 'hour', 'direction', 'orig', 'dest']).agg(total_seats=('seats', 'sum')).reset_index()

In [11]:
rg.head(10)

,date,day,hour,direction,orig,dest,total_seats
0,2023-06-05,Monday,7,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,64
1,2023-06-05,Monday,8,outbound,MASSY TGV,NANTES,32
2,2023-06-05,Monday,10,outbound,MASSY TGV,NANTES,5
3,2023-06-05,Monday,12,outbound,MASSY TGV,NANTES,1
4,2023-06-05,Monday,12,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,1
5,2023-06-05,Monday,13,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,6
6,2023-06-05,Monday,14,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,33
7,2023-06-05,Monday,15,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,134
8,2023-06-05,Monday,16,inbound,NANTES,PARIS MONTPARNASSE 1 ET 2,61
9,2023-06-05,Monday,16,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,2


In [16]:
for index, row in rg.iterrows():
    if row['direction'] == 'outbound':
        color = "blueviolet"
    else:
        color = "orangered"
    print(f"""
        {{
          day: {(row['date'] - (datetime.strptime('2023-06-04', '%Y-%m-%d')).date()).days+1},
          hour: {row['hour']},
          title: '{row['orig'][0:10]} > {row['dest'][0:10]}',
          alignment: 'left',
          color: '{color}'
        }},
    """)


        {
          day: 1,
          hour: 7,
          title: 'PARIS MONT > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 8,
          title: 'MASSY TGV > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 10,
          title: 'MASSY TGV > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 12,
          title: 'MASSY TGV > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 12,
          title: 'PARIS MONT > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 13,
          title: 'PARIS MONT > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          

In [23]:
start_dt = datetime.strptime('2023-06-01', '%Y-%m-%d')
end_dt = datetime.strptime('2023-06-10', '%Y-%m-%d')

delta = timedelta(days=1)

# store the dates between two dates in a list
dates = []

while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    dates.append(start_dt.date().strftime('%A %d/%m'))
    # increment start date by timedelta
    start_dt += delta

In [24]:
dates

['Thursday 01/06',
 'Friday 02/06',
 'Saturday 03/06',
 'Sunday 04/06',
 'Monday 05/06',
 'Tuesday 06/06',
 'Wednesday 07/06',
 'Thursday 08/06',
 'Friday 09/06',
 'Saturday 10/06']